In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import numpy as np
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
from MyDataLoader import *
mdl = MyDataLoader()
train_loader, val_loader, test_loader = mdl.get_dataloaders()
Nc, Ns, Nt = mdl.Nc, mdl.Ns, mdl.Nt
M = mdl.M
N2V = mdl.N2V
print(Nc, Ns, Nt)

15 212 16


# baseline mean model

In [4]:
tr_x = mdl.tr_X.numpy()
tr_c = mdl.tr_C.numpy()
tr_p = mdl.tr_PK.numpy()

te_x = mdl.te_X.numpy()
te_c = mdl.te_C.numpy()
te_p = mdl.te_PK.numpy()

tr_p = tr_p[:, -3:].sum(axis=1)
te_p = te_p[:, -3:].sum(axis=1)

In [5]:
_y_true = te_x
_y_pred = np.repeat(tr_x.mean(axis=0)[np.newaxis, :, :], te_x.shape[0], axis=0)

In [6]:
y_true = _y_true.reshape(-1)
y_pred = _y_pred.reshape(-1)

In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100 

mse = mean_squared_error(y_true, y_pred, squared=True)
rmse = mean_squared_error(y_true, y_pred, squared=False)
mae = mean_absolute_error(y_true, y_pred)
mape = MAPE(y_true, y_pred)

print(mse, rmse, mae, mape)
#16.380093 4.0472326 3.0643532 16.84139221906662

18.651743 4.3187666 3.228059 17.683635652065277


# baseline p based mean

In [8]:
_y_true = te_x
_y_pred = np.zeros(te_x.shape)
_y_pred[te_p == 0] = tr_x[tr_p == 0].mean(axis=0)
_y_pred[te_p == 1] = tr_x[tr_p == 1].mean(axis=0)

In [9]:
y_true = _y_true.reshape(-1)
y_pred = _y_pred.reshape(-1)

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100 

mse = mean_squared_error(y_true, y_pred, squared=True)
rmse = mean_squared_error(y_true, y_pred, squared=False)
mae = mean_absolute_error(y_true, y_pred)
mape = MAPE(y_true, y_pred)

print(mse, rmse, mae, mape)
#10.608762765719645 3.2571095722618306 2.4613136703392136 13.336050638582208

19.71723197989034 4.440408987907571 2.921866338726902 15.323500614148458


# Import LR, SVR, RFR

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

# LR

In [12]:
model = LinearRegression()
trXfeat = np.concatenate((tr_c.reshape(-1, Nc*Nt*2), tr_p.reshape(-1, 1)), axis=1)
teXfeat = np.concatenate((te_c.reshape(-1, Nc*Nt*2), te_p.reshape(-1, 1)), axis=1)
model.fit(trXfeat, tr_x.reshape(-1, Nt*Ns))
y_true = te_x.reshape(-1)
y_pred = model.predict(teXfeat).reshape(-1)

from sklearn.metrics import mean_squared_error, mean_absolute_error

def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100 

mse = mean_squared_error(y_true, y_pred, squared=True)
rmse = mean_squared_error(y_true, y_pred, squared=False)
mae = mean_absolute_error(y_true, y_pred)
mape = MAPE(y_true, y_pred)

print(mse, rmse, mae, mape)
#14.206448235066304 3.7691442311307624 2.8412641396162672 14.921364300336485

11.916018908073342 3.451958706020879 2.508077413272736 13.397856834977127


# SVR

In [13]:
from tqdm import notebook

In [14]:
y_trues = []
y_preds = []
for i in notebook.tqdm(range(Nt*Ns)):
    model = SVR()
    trXfeat = np.concatenate((tr_c.reshape(-1, Nc*Nt*2), tr_p.reshape(-1, 1)), axis=1)
    teXfeat = np.concatenate((te_c.reshape(-1, Nc*Nt*2), te_p.reshape(-1, 1)), axis=1)
    model.fit(trXfeat, tr_x.reshape(-1, Nt*Ns)[:, i])
    y_true = te_x.reshape(-1, Nt*Ns)[:, i].reshape(-1)
    y_pred = model.predict(teXfeat).reshape(-1)
    
    y_trues.append(y_true)
    y_preds.append(y_pred)
    
    if i%Nt != 0:
        continue
    y_true = np.array(y_trues).reshape(-1)
    y_pred = np.array(y_preds).reshape(-1)

    from sklearn.metrics import mean_squared_error, mean_absolute_error

    def MAPE(y_true, y_pred):
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100 

    mse = mean_squared_error(y_true, y_pred, squared=True)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)
    mape = MAPE(y_true, y_pred)

    print(mse, rmse, mae, mape)
    #14.206448235066304 3.7691442311307624 2.8412641396162672 14.921364300336485

1.6439090464135548 1.2821501653135465 1.0109589833103114 3.590897283944641
3.0383394997346604 1.743083331265221 1.3452262069673429 5.858040910830574
3.752135745251723 1.937043041662142 1.4965243444521534 6.987884456908445
3.9952560003318016 1.9988136482253172 1.5083750398066675 7.349272190266628
3.8482842331630556 1.9617044204372522 1.4715351368289211 7.040827876657138
6.8285306983692236 2.6131457476323865 1.8683807177571532 9.100281730378166
7.2430292422690785 2.691287655058277 1.9628639003940742 10.079990255025052
8.416283004384422 2.901083074367989 2.1348606871068294 10.939754520161353
9.64536112739994 3.1056981706856095 2.2843574978557624 12.897230311296267
9.068634899178218 3.0114174236027487 2.210083991285495 12.147405396494342
8.603470663936895 2.933167343323066 2.154244023680115 12.175030575235223
8.782212462776764 2.9634797895003038 2.2062101983311475 12.089811907649784
8.558973509825433 2.9255723388467825 2.1831356743646095 11.94676596965155
8.849080305623751 2.97474037617129

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/newmnt/miniconda3/envs/GIS/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-d340b337d619>", line 9, in <module>
    y_pred = model.predict(teXfeat).reshape(-1)
  File "/newmnt/miniconda3/envs/GIS/lib/python3.7/site-packages/sklearn/svm/_base.py", line 317, in predict
    return predict(X)
  File "/newmnt/miniconda3/envs/GIS/lib/python3.7/site-packages/sklearn/svm/_base.py", line 339, in _dense_predict
    cache_size=self.cache_size)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/newmnt/miniconda3/envs/GIS/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exce

KeyboardInterrupt: 

# RFR

In [15]:
model = RandomForestRegressor()
trXfeat = np.concatenate((tr_c.reshape(-1, Nc*Nt*2), tr_p.reshape(-1, 1)), axis=1)
teXfeat = np.concatenate((te_c.reshape(-1, Nc*Nt*2), te_p.reshape(-1, 1)), axis=1)
model.fit(trXfeat, tr_x.reshape(-1, Nt*Ns))
y_true = te_x.reshape(-1)
y_pred = model.predict(teXfeat).reshape(-1)

In [16]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100 

mse = mean_squared_error(y_true, y_pred, squared=True)
rmse = mean_squared_error(y_true, y_pred, squared=False)
mae = mean_absolute_error(y_true, y_pred)
mape = MAPE(y_true, y_pred)

print(mse, rmse, mae, mape)
#6.70	2.59	1.88	10.28

8.907374198352839 2.9845224405845636 2.0665147119237 11.100122827833939
